# Análise de Dados - UFPB

**Professor** : Marcus Carvalho

Alunos: Aldemir Carlos Alves e Ana Luísa Freitas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

desmatamento = (
    pd.read_csv('dados/municipio_bioma.csv'))
desmatamento

#geobr python para mapas e identificação de municípios

,ano,id_municipio,bioma,area_total,desmatado,vegetacao_natural,nao_vegetacao_natural,hidrografia
0,2012,1707207,Amazônia,0,61.9,-62.1,0.2,0.0
1,2004,1707207,Amazônia,0,60.0,-60.2,0.2,0.0
2,2015,1707207,Amazônia,0,61.9,-62.1,0.2,0.0
3,2021,1707207,Amazônia,0,61.9,-62.1,0.2,0.0
4,2017,1707207,Amazônia,0,61.9,-62.1,0.2,0.0
...,...,...,...,...,...,...,...,...
150323,2005,1100106,Amazônia,24831,1370.4,19462.8,3853.0,144.8
150324,2021,1100106,Amazônia,24831,1607.5,19225.7,3853.0,144.8
150325,2011,1100106,Amazônia,24831,1431.9,19401.3,3853.0,144.8
150326,2015,1100106,Amazônia,24831,1494.5,19338.7,3853.0,144.8
